# Optimization - Exercise

Take the SMA LRM process and define an optimization problem to get the highest possible recovery with > 95% purity. The limitations are:
- 10 seconds of loading time + 80 seconds of wash, then gradients of any shape
- 6000 seconds maximal process duration.
- 1000 mM maximal salt concentration

You will have one night with 12 cores per group to run the optimization.

In [ ]:
from CADETProcess.processModel import ComponentSystem
from CADETProcess.processModel import StericMassAction
from CADETProcess.processModel import Inlet, GeneralRateModel, Outlet, LumpedRateModelWithoutPores, TubularReactor
from CADETProcess.processModel import FlowSheet
from CADETProcess.processModel import Process
from CADETProcess.simulator import Cadet

component_system = ComponentSystem(['Salt', 'A', "B", "C"])

# Binding Model
binding_model = StericMassAction(component_system)
binding_model.is_kinetic = True
binding_model.adsorption_rate = [0, 1.3e-5, 5.59e-1, 9.5e-3]
binding_model.desorption_rate = [0, 1, 1, 1]
binding_model.characteristic_charge = [0, 6.9, 2.3, 5.8]
binding_model.steric_factor = [0, 10, 10.6, 11.83]
binding_model.capacity = 1.2e3

column = LumpedRateModelWithoutPores(component_system, name='column')
column.length = 0.014
column.total_porosity = 0.5
column.diameter = 0.01
column.axial_dispersion = 5.75e-7

pipe1 = TubularReactor(component_system, name="pipe1")
pipe1.length = 0.1
pipe1.diameter = 0.001
pipe1.axial_dispersion = 6e-6
pipe1.discretization.ncol = 50

pipe2 = TubularReactor(component_system, name="pipe2")
pipe2.length = 0.02
pipe2.diameter = 0.001
pipe2.axial_dispersion = 6e-6
pipe2.discretization.ncol = 50


column.binding_model = binding_model

column.q = [50, 0, 0, 0]
column.c = [50, 0, 0, 0]
pipe1.c = [50, 0, 0, 0]
pipe2.c = [50, 0, 0, 0]
column.volume

volumetric_flow_rate = 1.67e-8

inlet = Inlet(component_system, name='inlet')
inlet.flow_rate = volumetric_flow_rate

outlet = Outlet(component_system, name='outlet')

# Flow Sheet
flow_sheet = FlowSheet(component_system)

flow_sheet.add_unit(inlet, feed_inlet=True)
flow_sheet.add_unit(pipe1)
flow_sheet.add_unit(column)
flow_sheet.add_unit(pipe2)
flow_sheet.add_unit(outlet, product_outlet=True)

flow_sheet.add_connection(inlet, pipe1)
flow_sheet.add_connection(pipe1, column)
flow_sheet.add_connection(column, pipe2)
flow_sheet.add_connection(pipe2, outlet)

# Process
process = Process(flow_sheet, 'batch elution')

process.cycle_time = 6000

c_salt_load = 50
c_salt_gradient1_start = 86.89371792
c_salt_gradient1_end = 500
duration_gradient1 = 3127.53243
t_gradient1_start = 90
t_start_wash = 10
gradient_1_slope = (c_salt_gradient1_end - c_salt_gradient1_start)/(process.cycle_time - t_gradient1_start)

c_load = [c_salt_load, 1, 1, 1]

c_wash = [c_salt_load, 0, 0, 0]

c_gradient1_poly = [
    [c_salt_gradient1_start, gradient_1_slope, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
]

process.add_duration("grad1_duration", duration_gradient1)

process.add_event('load', 'flow_sheet.inlet.c', c_load, 0)
process.add_event('wash', 'flow_sheet.inlet.c', c_wash, t_start_wash)
process.add_event('grad1_start', 'flow_sheet.inlet.c', c_gradient1_poly, t_gradient1_start)

simulator = Cadet()
simulator.time_resolution = 5

simulation_results = simulator.simulate(process)
print(simulation_results.time_elapsed)

from CADETProcess.plotting import SecondaryAxis

sec = SecondaryAxis()
sec.components = ['Salt']
sec.y_label = '$c_{salt}$'

simulation_results.solution.outlet.outlet.plot(secondary_axis=sec)
import matplotlib.pyplot as plt

plt.tight_layout()

from CADETProcess.fractionation import FractionationOptimizer
from CADETProcess.optimization import OptimizationProblem

optimization_problem = OptimizationProblem(name="Optimize_Gradient")
optimization_problem.add_evaluation_object(process)
optimization_problem.add_evaluator(simulator)

from CADETProcess.performance import Recovery


fractionation_options = dict(
    purity_required=[0.00, 0.95, 0.00],
    components=["A", "B", "C"],
    )

fractionation_optimizer = FractionationOptimizer()
optimization_problem.add_evaluator(
    fractionation_optimizer,
    kwargs=fractionation_options
)

recovery = Recovery(ranking=1)
optimization_problem.add_objective(
    recovery,
    minimize=False,
    requires=[simulator, fractionation_optimizer]
)

from CADETProcess.plotting import SecondaryAxis

sec = SecondaryAxis()
sec.components = ['Salt']
sec.y_label = '$c_{salt}$'

simulation_results.solution.outlet.outlet.plot(secondary_axis=sec)
import matplotlib.pyplot as plt

plt.tight_layout()

# starting concentration 2nd => slope * duration
# starting time 2nd => duration + starting time 1st
# slope 2nd = func()
process.add_event("grad2_start", "flow_sheet.inlet.c", [[0, 0, 0, 0], ]*4, time=200)

# gradient1 start time
optimization_problem.add_variable(
    "grad1_start_time",
    parameter_path="grad1_start.time",
    lb=90,
    ub=100
)

optimization_problem.add_variable(
    "grad1_start_conc",
    parameter_path="grad1_start.state",
    lb=50, ub=500,
    indices=(0,0)
)

# gradient1 slope
optimization_problem.add_variable(
    "gradient1_slope",
    lb=0.001, ub=10, indices=(0, 1),
    parameter_path='grad1_start.state'
)

optimization_problem.add_variable(
    'grad1_duration.time',
    lb=120,
    ub=5790
)

# gradient2 start time
var = optimization_problem.add_variable(
    "grad2_start.time",
    lb=500,
    ub=900,
)

# gradient2 start concentration
var = optimization_problem.add_variable(
    "gradient2_start_conc", lb=1, ub=1e5, indices=(0, 0),
    parameter_path='grad2_start.state'
)

# gradient2 slope
optimization_problem.add_variable(
    "gradient2_slope", lb=-100, ub=1e6, indices=(0, 1),
    parameter_path='grad2_start.state'
)



optimization_problem.check_linear_equality_constraints([10, 50, 0.1, 100, 110, 100, 0.1])
optimization_problem.variable_names

optimization_problem.variable_names

def calculate_salt_concentration_after_gradient(x):
    salt_concentration_after_gradient_1 = x[1] + x[2] * x[3]
    return salt_concentration_after_gradient_1

optimization_problem.add_nonlinear_constraint(
    calculate_salt_concentration_after_gradient,
    bounds=1000,
    comparison_operator="le",
    evaluation_objects=None,
    name="salt_after_gradient_1",
)

optimization_problem.check_nonlinear_constraints(x=[100, 50, 0.1, 200, 700, 1, 0])

optimization_problem.variable_names

optimization_problem.add_variable_dependency(
    "gradient2_start_conc",
    ["grad1_start_conc", "gradient1_slope", "grad1_duration.time"],
    transform=lambda start_conc, slope, duration: start_conc + slope * duration
)

optimization_problem.add_variable_dependency(
    dependent_variable="gradient2_slope",
    independent_variables=["grad1_start_conc", "gradient1_slope", "grad1_duration.time"],
    transform=lambda start_conc, slope, duration:
    (1000 - (start_conc + slope * duration)) / (process.cycle_time - 90 - duration)
)

optimization_problem.add_variable_dependency(
    "grad2_start.time",
    ["grad1_start_time", "grad1_duration.time" ],
    transform=lambda starttime, duration: starttime+duration
)

def callback(fractionation, individual, evaluation_object, callbacks_dir):
    fractionation.plot_fraction_signal(
        file_name=f'{callbacks_dir}/{individual.id[:10]}_{evaluation_object}_fractionation.png',
        show=False
    )

optimization_problem.add_callback(
    callback, requires=[simulator, fractionation_optimizer]
)

def callback_sim(simulation_results, individual, callbacks_dir='./'):
    sec = SecondaryAxis()
    sec.components = ['Salt']
    sec.y_label = '$c_{salt}$'

    simulation_results.solution.outlet.outlet.plot(
        secondary_axis=sec,
        show=False,
        file_name=f'{callbacks_dir}/{individual.id[:10]}.png'
    )

optimization_problem.add_callback(
    callback_sim, requires=[simulator, ]
)

from CADETProcess.optimization import U_NSGA3

optimizer = U_NSGA3()
optimizer.n_cores = 6
optimizer.pop_size = 60
optimizer.n_max_gen = 4

# optimization_results = optimizer.optimize(
#     optimization_problem
# )